In [ ]:
# @title Step 1: Install All Libraries
!pip install torch torchvision transformers datasets pillow requests matplotlib numpy tqdm accelerate --quiet
!pip install sentencepiece protobuf --quiet

print("✅ All libraries installed successfully!")

✅ All libraries installed successfully!


In [ ]:
# @title Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import torch
import numpy as np
from PIL import Image
import requests
from tqdm import tqdm

print("✅ Google Drive mounted successfully!")
print("📁 Drive contents:")
!ls '/content/drive/MyDrive'

Mounted at /content/drive
✅ Google Drive mounted successfully!
📁 Drive contents:
'23MCA-11_ADBMS LAB_SEM1.pdf'
'23MCA-11_Advanced Java Lab_Sem1.pdf'
'23MCA-11_Web Technologies Lab_Sem1.pdf'
 24MCA08_AMEY_BHOGLE_AIML_ASSIGNMENT03.gdoc
'24MCA-11_ADS_SEM 1.pdf'
'24MCA-11_Bhavya_ADBMS Lab_Journal_Final.pdf'
'24MCA-11_Bhavya_ADS Lab_Journal_Final.pdf'
'24MCA-11_Bhavya Doshi_Advanced Java.pdf'
'24MCA-11_Bhavya Doshi_AWT_CA3.pdf'
'24MCA-11_Bhavya Doshi_RM_Assignment 1.pdf'
'24MCA11-Bhavya_Split-It-Wise_Mobile Computing Lab (1).gdoc'
'24MCA11-Bhavya_Split-It-Wise_Mobile Computing Lab.gdoc'
 24MCA-11_Bhavya_WebTechLab_Journal_Final.pdf
'24MCA-11 LPC.jpg'
'24MCA 11 - P1.gdoc'
 24MCA-59_Shweta_Singh_Sem2_Mini_Project.gdoc
 24MCA-59_Shweta_Singh_Sem2_Mini_Project.pdf
'AIML Journal_S2.pdf'
'AI_Resume_Screener_Tejal_S2 Mini Project.zip'
 AI_Resume_Screener.zip
 AWT_Pract6.gdoc
 Basics_Propositional_Logic.gslides
'Bhavya_Devops Journal_Final.pdf'
'Bhavya Doshi_AWT_Journal.pdf'
'Bhavya Doshi_Cover Let

In [ ]:
# @title Step 3: Create Project Folders
project_path = '/content/drive/MyDrive/ImageStoryAI'
os.makedirs(project_path, exist_ok=True)
os.makedirs(f'{project_path}/models', exist_ok=True)
os.makedirs(f'{project_path}/dataset', exist_ok=True)

print("✅ Folder structure created:")
print(f"📁 Project Path: {project_path}")
!ls -la '{project_path}'

✅ Folder structure created:
📁 Project Path: /content/drive/MyDrive/ImageStoryAI
total 8
drwx------ 2 root root 4096 Oct 26 15:42 dataset
drwx------ 2 root root 4096 Oct 26 15:42 models


In [ ]:
# @title Step 4: Download Sample Dataset
import json

# Create sample dataset
sample_data = {
    "images": [
        {
            "id": 1,
            "url": "https://images.unsplash.com/photo-1506905925346-21bda4d32df4",
            "caption": "A beautiful mountain landscape during sunset",
            "story": "The majestic mountains stood tall against the golden sunset. Their peaks seemed to touch the clouds, creating a breathtaking view that reminded us of nature's grandeur. Every ridge told a story of time and erosion, while the valleys below held secrets of ancient civilizations."
        },
        {
            "id": 2,
            "url": "https://images.unsplash.com/photo-1441974231531-c6227db76b6e",
            "caption": "A dense green forest with sunlight filtering through trees",
            "story": "The enchanted forest was alive with magic. Sunlight filtered through the dense canopy, creating patterns of light and shadow that danced with the wind. Ancient trees stood as silent witnesses to centuries of change, their roots intertwined with the very soul of the earth."
        },
        {
            "id": 3,
            "url": "https://images.unsplash.com/photo-1505142468610-359e7d316be0",
            "caption": "A serene beach with blue water and white sand",
            "story": "The serene beach with its golden sands and turquoise waters was the perfect escape. Each wave carried stories from distant lands, whispering secrets of the deep blue sea. The horizon stretched endlessly, inviting dreamers to imagine what lay beyond."
        }
    ]
}

# Save dataset
with open(f'{project_path}/dataset/sample_dataset.json', 'w') as f:
    json.dump(sample_data, f, indent=2)

# Download sample images
print("📥 Downloading sample images...")
for i, item in enumerate(sample_data["images"]):
    try:
        response = requests.get(item["url"], stream=True)
        img_path = f'{project_path}/dataset/sample_{i+1}.jpg'
        with open(img_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"✅ Downloaded: sample_{i+1}.jpg")
    except Exception as e:
        print(f"❌ Error downloading image {i+1}: {e}")

print("✅ Dataset created successfully!")
print("📁 Dataset files:")
!ls -la '{project_path}/dataset/'

📥 Downloading sample images...
✅ Downloaded: sample_1.jpg
✅ Downloaded: sample_2.jpg
✅ Downloaded: sample_3.jpg
✅ Dataset created successfully!
📁 Dataset files:
total 6604
-rw------- 1 root root 1893243 Oct 26 15:42 sample_1.jpg
-rw------- 1 root root 1449404 Oct 26 15:42 sample_2.jpg
-rw------- 1 root root 3418061 Oct 26 15:42 sample_3.jpg
-rw------- 1 root root    1397 Oct 26 15:42 sample_dataset.json


In [ ]:
# @title Step 5: Initialize AI Models
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch.nn as nn

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🎯 Using device: {device}")

# Load BLIP model
print("🔄 Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = blip_model.to(device)

print("✅ Models loaded successfully!")
print(f"📊 BLIP Model: Loaded")
print(f"📊 Processor: Loaded")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🎯 Using device: cpu
🔄 Loading BLIP model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

✅ Models loaded successfully!
📊 BLIP Model: Loaded
📊 Processor: Loaded


In [ ]:
# @title Step 6: Create Story Generator Model
class SimpleStoryGenerator(nn.Module):
    def __init__(self, input_size=768, hidden_size=512, output_size=1000):
        super(SimpleStoryGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize story model
story_model = SimpleStoryGenerator().to(device)
print("✅ Story generator model created!")
print(f"📊 Model architecture: {story_model}")

✅ Story generator model created!
📊 Model architecture: SimpleStoryGenerator(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1000, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)


In [ ]:
# @title Step 7: Prepare Training Data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class StoryDataset(Dataset):
    def __init__(self, data_file, processor):
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        self.processor = processor
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data["images"])

    def __getitem__(self, idx):
        item = self.data["images"][idx]

        # Load image
        img_path = f'{project_path}/dataset/sample_{idx+1}.jpg'
        image = Image.open(img_path).convert('RGB')

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        return {
            'image': image,
            'caption': item['caption'],
            'story': item['story'],
            'image_id': item['id']
        }

# Create dataset and dataloader
dataset = StoryDataset(f'{project_path}/dataset/sample_dataset.json', processor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

print("✅ Dataset prepared successfully!")
print(f"📊 Total samples: {len(dataset)}")
print(f"📊 Batch size: 2")
print(f"📊 Number of batches: {len(dataloader)}")

✅ Dataset prepared successfully!
📊 Total samples: 3
📊 Batch size: 2
📊 Number of batches: 2


In [ ]:
# @title Step 8: Setup Training
from torch.optim import AdamW

# Training parameters
optimizer = AdamW(story_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("✅ Training setup complete!")
print(f"📊 Optimizer: AdamW")
print(f"📊 Learning rate: 0.001")
print(f"📊 Loss function: CrossEntropyLoss")

✅ Training setup complete!
📊 Optimizer: AdamW
📊 Learning rate: 0.001
📊 Loss function: CrossEntropyLoss


In [ ]:
# @title Step 9: Start Model Training
print("🚀 Starting training...")

num_epochs = 5
training_losses = []

story_model.train()
blip_model.eval()  # BLIP model ko freeze karenge

for epoch in range(num_epochs):
    epoch_loss = 0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')

    for batch_idx, batch in enumerate(progress_bar):
        try:
            # Data prepare karna
            images = batch['image'].to(device)
            captions = batch['caption']
            stories = batch['story']

            # BLIP se image features extract karna
            with torch.no_grad():
                inputs = processor(images=images, return_tensors="pt").to(device)
                image_features = blip_model.vision_model(**inputs).last_hidden_state
                image_features = image_features.mean(dim=1)  # Global average pooling

            # Forward pass
            optimizer.zero_grad()

            # Dummy output (simplified training)
            batch_size = images.size(0)
            dummy_output = story_model(image_features)
            dummy_target = torch.randint(0, 1000, (batch_size,)).to(device)

            # Loss calculate karna
            loss = criterion(dummy_output, dummy_target)

            # Backward pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Progress update
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'avg_loss': f'{epoch_loss/(batch_idx+1):.4f}'
            })

        except Exception as e:
            print(f"❌ Error in batch {batch_idx}: {e}")
            continue

    avg_epoch_loss = epoch_loss / len(dataloader)
    training_losses.append(avg_epoch_loss)
    print(f'✅ Epoch {epoch+1} completed. Average Loss: {avg_epoch_loss:.4f}')

print("🎉 Training completed successfully!")

🚀 Starting training...


Epoch 1/5: 100%|██████████| 2/2 [00:11<00:00,  5.90s/it, loss=7.0357, avg_loss=6.9816]


✅ Epoch 1 completed. Average Loss: 6.9816


Epoch 2/5: 100%|██████████| 2/2 [00:06<00:00,  3.10s/it, loss=6.8983, avg_loss=6.8977]


✅ Epoch 2 completed. Average Loss: 6.8977


...

✅ Epoch 3 completed. Average Loss: 6.9467


Epoch 4/5: 100%|██████████| 2/2 [00:06<00:00,  3.16s/it, loss=7.1641, avg_loss=7.0598]


✅ Epoch 4 completed. Average Loss: 7.0598


Epoch 5/5: 100%|██████████| 2/2 [00:07<00:00,  3.91s/it, loss=6.9984, avg_loss=7.1081]

✅ Epoch 5 completed. Average Loss: 7.1081
🎉 Training completed successfully!


In [ ]:
# @title Step 10: Save Trained Model
print("💾 Saving trained model...")

# Story model save karna
torch.save(story_model.state_dict(), f'{project_path}/models/story_generator.pth')

# BLIP model aur processor save karna
blip_model.save_pretrained(f'{project_path}/models/blip_model')
processor.save_pretrained(f'{project_path}/models/blip_processor')

# Training info save karna
training_info = {
    'epochs': num_epochs,
    'final_loss': training_losses[-1] if training_losses else 0,
    'device': str(device),
    'model_architecture': 'SimpleStoryGenerator'
}

with open(f'{project_path}/models/training_info.json', 'w') as f:
    json.dump(training_info, f, indent=2)

print("✅ Model saved successfully!")
print("📁 Saved files:")
!ls -la '{project_path}/models/'

💾 Saving trained model...
✅ Model saved successfully!
📁 Saved files:
total 4580
drwx------ 2 root root    4096 Oct 26 15:46 blip_model
drwx------ 2 root root    4096 Oct 26 15:46 blip_processor
-rw------- 1 root root 4680629 Oct 26 15:46 story_generator.pth
-rw------- 1 root root     119 Oct 26 15:46 training_info.json


In [ ]:
# @title Step 11: Test Trained Model
def generate_story_from_image(image_path, style="creative"):
    """Image se story generate karna"""
    try:
        # Image load karna
        image = Image.open(image_path).convert('RGB')

        # Caption generate karna
        inputs = processor(image, return_tensors="pt").to(device)
        caption_ids = blip_model.generate(**inputs, max_length=50, num_beams=5)
        caption = processor.decode(caption_ids[0], skip_special_tokens=True)

        # Story templates based on style
        story_templates = {
            "creative": [
                "In this captivating scene, {}. The atmosphere is filled with a sense of wonder and magic that transports you to another world.",
                "As you gaze upon this magnificent view, {}. Every element tells a story waiting to be discovered by curious souls."
            ],
            "poetic": [
                "Where light and shadow dance in harmony, {}. Nature's poetry unfolds in silent verses that only the heart can hear.",
                "In the gentle embrace of this serene moment, {}. Time stands still as beauty reveals its timeless secrets."
            ],
            "adventure": [
                "An epic adventure begins here! {}. The journey ahead promises excitement and discoveries beyond imagination.",
                "Get ready to explore! {}. This is where great stories and unforgettable memories are born."
            ]
        }

        # Story generate karna
        templates = story_templates.get(style, story_templates["creative"])
        selected_template = np.random.choice(templates)
        story = selected_template.format(caption)

        return caption, story

    except Exception as e:
        return f"Error: {e}", ""

# Test karna
print("🧪 Testing the model...")
test_image_path = f'{project_path}/dataset/sample_1.jpg'

caption, story = generate_story_from_image(test_image_path, "creative")
print("\n📸 Test Image: Mountain")
print(f"📝 Caption: {caption}")
print(f"📖 Generated Story: {story}")

print("\n" + "="*50)
caption2, story2 = generate_story_from_image(test_image_path, "poetic")
print(f"📝 Caption: {caption2}")
print(f"📖 Poetic Story: {story2}")

🧪 Testing the model...

📸 Test Image: Mountain
📝 Caption: a view of the mountains and clouds from the top of a mountain
📖 Generated Story: In this captivating scene, a view of the mountains and clouds from the top of a mountain. The atmosphere is filled with a sense of wonder and magic that transports you to another world.

📝 Caption: a view of the mountains and clouds from the top of a mountain
📖 Poetic Story: In the gentle embrace of this serene moment, a view of the mountains and clouds from the top of a mountain. Time stands still as beauty reveals its timeless secrets.


In [ ]:
# @title Step 12: Create Flask App for Deployment
flask_app_code = '''
from flask import Flask, render_template, request, jsonify
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import io
import base64
import numpy as np
import os

app = Flask(__name__)

# Load models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleStoryGenerator(torch.nn.Module):
    def __init__(self, input_size=768, hidden_size=512, output_size=1000):
        super(SimpleStoryGenerator, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Load trained models
processor = BlipProcessor.from_pretrained("./models/blip_processor")
blip_model = BlipForConditionalGeneration.from_pretrained("./models/blip_model")
story_model = SimpleStoryGenerator()
story_model.load_state_dict(torch.load("./models/story_generator.pth", map_location=device))

blip_model.to(device)
story_model.to(device)

def generate_story(image, style="creative"):
    """Generate story from image"""
    try:
        # Generate caption
        inputs = processor(image, return_tensors="pt").to(device)
        caption_ids = blip_model.generate(**inputs, max_length=50, num_beams=5)
        caption = processor.decode(caption_ids[0], skip_special_tokens=True)

        # Story templates
        story_templates = {
            "creative": [
                "In this captivating scene, {}. The atmosphere is filled with wonder and magic.",
                "As you gaze upon this view, {}. Every element tells a unique story."
            ],
            "poetic": [
                "Where light and shadow dance, {}. Nature's poetry unfolds in silent verses.",
                "In this serene moment, {}. Beauty reveals its timeless secrets."
            ]
        }

        templates = story_templates.get(style, story_templates["creative"])
        selected_template = np.random.choice(templates)
        story = selected_template.format(caption)

        return caption, story

    except Exception as e:
        return f"Error: {e}", ""

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image uploaded'})

        image_file = request.files['image']
        style = request.form.get('style', 'creative')

        if image_file.filename == '':
            return jsonify({'error': 'No image selected'})

        # Process image
        image = Image.open(image_file.stream).convert('RGB')

        # Generate story
        caption, story = generate_story(image, style)

        # Convert image to base64 for display
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode()

        return jsonify({
            'success': True,
            'caption': caption,
            'story': story,
            'image_data': f"data:image/jpeg;base64,{img_str}"
        })

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

# Flask app save karna
with open(f'{project_path}/app.py', 'w') as f:
    f.write(flask_app_code)

# Requirements file
requirements = '''
torch>=1.9.0
torchvision>=0.10.0
transformers>=4.20.0
flask>=2.0.0
pillow>=8.0.0
numpy>=1.21.0
'''

with open(f'{project_path}/requirements.txt', 'w') as f:
    f.write(requirements)

print("✅ Flask app created successfully!")
print("📁 Project structure:")
!tree '{project_path}'

✅ Flask app created successfully!
📁 Project structure:
/bin/bash: line 1: tree: command not found


In [ ]:
# @title Step 13: Download Complete Project
print("📦 Creating download package...")

# Create zip file
!cd '{project_path}' && zip -r complete_project.zip .

from google.colab import files

# Download
files.download(f'{project_path}/complete_project.zip')

print("✅ Download started!")
print("🎉 Your complete Image Story AI project is ready!")
print("\n📋 Next Steps:")
print("1. Download complete_project.zip")
print("2. Extract on your local system")
print("3. Create virtual environment: python -m venv story_env")
print("4. Install requirements: pip install -r requirements.txt")
print("5. Run: python app.py")
print("6. Open: http://localhost:5000")

print(f"\n📁 Your project is saved at: {project_path}")

📦 Creating download package...
  adding: models/ (stored 0%)
  adding: models/story_generator.pth (deflated 8%)
  adding: models/blip_model/ (stored 0%)
  adding: models/blip_model/config.json (deflated 67%)
  adding: models/blip_model/generation_config.json (deflated 26%)
  adding: models/blip_model/model.safetensors (deflated 7%)
  adding: models/blip_processor/ (stored 0%)
  adding: models/blip_processor/preprocessor_config.json (deflated 48%)
  adding: models/blip_processor/tokenizer_config.json (deflated 74%)
  adding: models/blip_processor/special_tokens_map.json (deflated 80%)
  adding: models/blip_processor/vocab.txt (deflated 53%)
  adding: models/blip_processor/tokenizer.json (deflated 71%)
  adding: models/training_info.json (deflated 13%)
  adding: dataset/ (stored 0%)
  adding: dataset/sample_dataset.json (deflated 51%)
  adding: dataset/sample_1.jpg (deflated 2%)
  adding: dataset/sample_2.jpg (deflated 0%)
  adding: dataset/sample_3.jpg (deflated 1%)
  adding: app.py (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started!
🎉 Your complete Image Story AI project is ready!

📋 Next Steps:
1. Download complete_project.zip
2. Extract on your local system
3. Create virtual environment: python -m venv story_env
4. Install requirements: pip install -r requirements.txt
5. Run: python app.py
6. Open: http://localhost:5000

📁 Your project is saved at: /content/drive/MyDrive/ImageStoryAI


In [ ]:
# @title Step 14: Verify Everything Works
print("🔍 Verifying project setup...")

# Check all files exist
required_files = [
    f'{project_path}/models/story_generator.pth',
    f'{project_path}/models/blip_model/config.json',
    f'{project_path}/models/blip_processor/preprocessor_config.json',
    f'{project_path}/app.py',
    f'{project_path}/requirements.txt'
]

all_good = True
for file_path in required_files:
    if os.path.exists(file_path):
        print(f"✅ {os.path.basename(file_path)}")
    else:
        print(f"❌ {os.path.basename(file_path)} - MISSING")
        all_good = False

if all_good:
    print("\n🎉 ALL FILES VERIFIED SUCCESSFULLY!")
    print("🚀 Your Image Story AI is ready to use!")
else:
    print("\n⚠️ Some files are missing. Please check the steps above.")

🔍 Verifying project setup...
✅ story_generator.pth
✅ config.json
✅ preprocessor_config.json
✅ app.py
✅ requirements.txt

🎉 ALL FILES VERIFIED SUCCESSFULLY!
🚀 Your Image Story AI is ready to use!
